In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models

In [2]:
dataset_path = "./fall_dataset/"

## PREPROCESSING

In [3]:

# Function to check if a label contains more than one class
def contains_multiple_classes(label_file_path):
    with open(label_file_path, "r") as f:
        label_lines = f.readlines()
        return len(label_lines) > 1

# Function to delete a file
def delete_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    else:
        print(f"File not found: {file_path}")

# Load the dataset and labels
def load_dataset():
    images = []
    labels = []

    image_dir = os.path.join(dataset_path, "images","train")
    label_dir = os.path.join(dataset_path, "labels","train")

    # Set the desired image size (you can adjust this as needed)
    image_size = (256, 256)

    for filename in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            img_resized = cv2.resize(img, image_size)

            images.append(img_resized)

            label_filename = filename[:-4] + ".txt"
            label_filepath = os.path.join(label_dir, label_filename)

            # Check if the label contains more than one class
            
            with open(label_filepath, "r") as f:
                label_values = [float(value) for value in f.readline().split()]

                # Convert class 2 to class 1
                if label_values[0] == 2:
                    label_values[0] = 1

                # Convert multi-class to binary labels
                if label_values[0] == 1:
                    labels.append([1.0])  # Not Fallen class
                else:
                    labels.append([0.0])  # Fallen class

    return np.array(images), np.array(labels)

# Load the dataset
images, labels = load_dataset()

In [4]:
# Preprocess the dataset
def preprocess_dataset(images, labels, test_size=0.2):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42)

    # Normalize pixel values to the range [0, 1]
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    # Extract class labels from bounding box information
    y_train_classes = y_train[:, 0].astype('int')
    y_test_classes = y_test[:, 0].astype('int')

    return X_train, X_test, y_train_classes, y_test_classes

# Preprocess the dataset
X_train, X_test, y_train_classes, y_test_classes = preprocess_dataset(images, labels, test_size=0.2)

In [5]:
# Data Augmentation
def augment_dataset(X_train):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    datagen.fit(X_train)

    return datagen

# Data Augmentation
data_augmentor = augment_dataset(X_train)

## MODEL CREATION

In [6]:
# Define the CNN model for fall detection
def create_fall_detection_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Set the input shape and number of classes
input_shape = X_train[0].shape
num_classes = 2  # Two classes: Fall and Non-Fall

# Create the model
model = create_fall_detection_model(input_shape, num_classes)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

## MODEL TRAINING

In [46]:
# Train the model
epochs = 100
batch_size = 32

model.fit(data_augmentor.flow(X_train, y_train_classes, batch_size=batch_size),
          steps_per_epoch=len(X_train) // batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test_classes))


Epoch 1/500
8/8 [==============================] - 2s 168ms/step - loss: 0.4612 - accuracy: 0.7734 - val_loss: 0.6341 - val_accuracy: 0.7059
Epoch 2/500
8/8 [==============================] - 1s 164ms/step - loss: 0.3959 - accuracy: 0.8270 - val_loss: 0.6953 - val_accuracy: 0.6765
Epoch 3/500
8/8 [==============================] - 1s 182ms/step - loss: 0.4297 - accuracy: 0.7975 - val_loss: 0.6644 - val_accuracy: 0.7059
Epoch 4/500
8/8 [==============================] - 1s 161ms/step - loss: 0.4470 - accuracy: 0.7722 - val_loss: 0.6362 - val_accuracy: 0.7059
Epoch 5/500
8/8 [==============================] - 1s 189ms/step - loss: 0.4474 - accuracy: 0.8101 - val_loss: 0.6419 - val_accuracy: 0.6765
Epoch 6/500
8/8 [==============================] - 2s 195ms/step - loss: 0.4194 - accuracy: 0.8017 - val_loss: 0.6681 - val_accuracy: 0.6765
Epoch 7/500
8/8 [==============================] - 1s 195ms/step - loss: 0.4172 - accuracy: 0.8143 - val_loss: 0.6833 - val_accuracy: 0.6765
Epoch 8/500
8

In [47]:
evaluation = model.evaluate(X_test, y_test_classes)

# Print the evaluation metrics (accuracy and loss)
print(f"Validation Accuracy: {evaluation[1]:.2f}")
print(f"Validation Loss: {evaluation[0]:.4f}")

3/3 [==============================] - 0s 16ms/step - loss: 1.1038 - accuracy: 0.7353
Validation Accuracy: 0.74
Validation Loss: 1.1038


In [48]:
model.save("./Fall_detect_model_0.74.h5")